In [ ]:
# Create or update conda environment
# !conda env create -f ../environment.yaml
# !conda env update -f ../environment.yaml

In [ ]:
# Activate conda environment
!conda activate amadeus-ex-machina

In [ ]:
# System imports
import sys
import os

# Add the parent directory of 'notebooks' to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move one level up
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Test if the path is correctly added
print(f"Parent directory added to sys.path: {parent_dir}")

# Import packages
import json
import torchaudio
import torch
import torch.optim as optim
import torch.nn as nn
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from torch.utils.data import Dataset, DataLoader

# Import models
from models.CRNN import CRNNModel
from models.CNN import CNNModel
from models.RNN import RNNModel
from models.AudioDataset import ChordDataset

json_file = "../datagen/chords/chord_ref.json"
audio_dir = "../datagen/chords/midi/"

In [ ]:
# Data preprocessing

# Load the JSON metadata
with open(json_file, "r") as f:
    metadata = json.load(f)

# Create a mapping for chord classes to integers
chord_classes = sorted({value["chord_class"] for value in metadata.values()})
chord_class_to_idx = {chord: idx for idx, chord in enumerate(chord_classes)}
print("Chord Class to Index Mapping:", chord_class_to_idx)

# Create Dataset and DataLoader
dataset = ChordDataset(metadata, audio_dir, chord_class_to_idx)

# Split into training and validation datasets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Move model to the selected device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Get the number of classes
num_classes = len(chord_class_to_idx)

In [ ]:
# Modular training and validation function

def training_validation(model):
    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training Loop
    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        # Loop over the batches in the training dataset
        for spectrograms, labels in train_loader:
            # Move data to the selected device
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            # Zero gradients from the previous step
            optimizer.zero_grad()

            # Forward pass
            outputs = model(spectrograms)

            # Compute loss
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()

            # Update model weights
            optimizer.step()

            total_loss += loss.item()

        # Print training loss for the current epoch
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

        # Evaluate on validation data after every epoch
        model.eval()  # Set model to evaluation mode
        correct = 0
        total = 0
        with torch.no_grad():  # No need to compute gradients during evaluation
            for spectrograms, labels in val_loader:
                spectrograms, labels = spectrograms.to(device), labels.to(device)
                outputs = model(spectrograms)
                _, predicted = torch.max(outputs, 1)  # Get predicted class

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f"Validation Accuracy: {val_accuracy:.2f}%")

In [ ]:
# Initialize CRNN model
crnn_model = CRNNModel(input_channels=2, num_classes=num_classes, hidden_size=128).to(device)
training_validation(crnn_model)

In [ ]:
# Initialize CNN model
cnn_model = CNNModel(input_channels=2, num_classes=num_classes).to(device)
training_validation(cnn_model)

In [ ]:
# Calculate the flattened size of the spectrograms
for spectrograms, _ in train_loader:
    rnn_input_size = spectrograms.view(spectrograms.size(0), -1).size(1)
    break

# Initialize the RNN model
rnn_model = RNNModel(input_size=rnn_input_size, hidden_size=128, output_size=num_classes).to(device)
training_validation(rnn_model)